In [ ]:
import fig02 as src
import pandas as pd

signature_list = ["nps", "siips", "na-gen", "na-therm", "na-mech", "na-sound", "na-vis"]

cbf = (
    pd.read_csv("../../results/cbf-ps-agg.csv")
    .query("ses == 'mri1'")
    .pipe(src.drop_outliers, "cbfQEI")
    .groupby("sub")
    .filter(lambda df: df["task"].nunique() == 2)
    .pipe(src.scale, signature_list)
    .melt(id_vars=["sub", "task"], value_vars=signature_list)
)


In [ ]:
import pingouin as pg

(
    cbf.groupby("variable")
    .apply(lambda df: df.pairwise_tests(dv="value", within="task", subject="sub"))
    .assign(fdr=lambda df: pg.multicomp(df["p-unc"].values, alpha=0.05, method="fdr")[0])
)


In [ ]:
import seaborn as sns

cmap = sns.color_palette(["#313695", "#D62F26"], n_colors=2)
ax = sns.pointplot(
    data=cbf,
    x="variable",
    y="value",
    hue="task",
    errorbar="se",
    palette=cmap,
    order=signature_list,
    join=False,
)
ax.axhline(0, linestyle="--", lw=0.5, color="k")
ax.set_xticklabels(ax.get_xticklabels(), rotation=15)
ax.set_ylabel("brain pattern response (zscore)")
ax.set_xlabel("")
fig = ax.get_figure()
fig.savefig("../../figures/sup01.svg")
